<a href="https://colab.research.google.com/github/quangnguyen-james/Datamining-Technical/blob/main/FP_Growth_method_by_manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CÁC BƯỚC THỰC HIỆN THUẬT TOÁN FB-GROWTH
1. Quét lần 1 để tính support từng item.
2. Loại bỏ item < minSupport.
3. Sắp xếp lại item trong từng transaction theo tần suất giảm dần.
4. Xây FP-tree.
5. Duyệt FP-tree để tạo ra các frequent pattern (dùng đệ quy + conditional FP-tree).

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
import random

In [72]:
transactions =[['Bánh mì', 'Sữa', 'Bơ'],
    ['Bánh mì', 'Bơ'],
    ['Sữa', 'Bơ'],
    ['Bánh mì', 'Sữa'],
    ['Bánh mì', 'Sữa', 'Bơ']]
minSupport = 0.5
min_support_count = int(minSupport * len(transactions))
minConfidence = 0.7

5


## Bước 1: Đếm tần suất item
**Mục đích**: Đếm số lượng items xuất hiện trong tất cả transactions

In [62]:
from collections import defaultdict

def get_item_frequency(transactions):
    freq = defaultdict(int)
    for trans in transactions:
        for item in trans:
            freq[item] += 1
    return freq
item_frequencies = get_item_frequency(transactions)
print(item_frequencies)

defaultdict(<class 'int'>, {'Bánh mì': 4, 'Sữa': 4, 'Bơ': 4})


## Bước 2: Lọc theo minSupport
**Mục đích:** Tìm và loại bỏ tất cả các items nhỏ hơn hoặc bằng minSupport

In [71]:
def filter_items(freq, min_support, num_transactions):
    return {item for item, count in freq.items() if count / num_transactions >= min_support}
filtered_items = filter_items(item_frequencies, minSupport, len(transactions))
print(filtered_items)

{'Sữa', 'Bánh mì', 'Bơ'}


## Bước 3: Sắp xếp transaction theo tần suất
**Mục đích:** Sắp xếp lại từng transaction theo thứ tự giảm dần của mỗi item

In [64]:
def sort_transactions(transactions, freq):
    sorted_trans = []
    for trans in transactions:
        filtered = [item for item in trans if item in freq]
        # Sắp theo tần suất giảm dần
        sorted_items = sorted(filtered, key=lambda x: (-freq[x], x))
        sorted_trans.append(sorted_items)
    return sorted_trans
sorted_transactions = sort_transactions(transactions, item_frequencies)
print(sorted_transactions)

[['Bánh mì', 'Bơ', 'Sữa'], ['Bánh mì', 'Bơ'], ['Bơ', 'Sữa'], ['Bánh mì', 'Sữa'], ['Bánh mì', 'Bơ', 'Sữa']]


## Bước 4: Xây dựng cây FB-tree (Cây gốc)

In [65]:
class FPTreeNode:
    def __init__(self, item, count=1):
        self.item = item
        self.count = count
        self.parent = None
        self.children = {}
        self.link = None  # Liên kết cùng item

    def increment(self, count):
        self.count += count

    def display(self, indent=0):
        print('  ' * indent + f"{self.item}:{self.count}")
        for child in self.children.values():
            child.display(indent + 1)

#  Xây dựng cây
def build_fp_tree(sorted_transactions, freq):
    header_table = {item: [freq[item], None] for item in freq}
    root = FPTreeNode('Null', 1)

    for trans in sorted_transactions:
        current_node = root
        for item in trans:
            if item in current_node.children:
                current_node.children[item].increment(1)
            else:
                new_node = FPTreeNode(item, 1)
                new_node.parent = current_node
                current_node.children[item] = new_node

                # Liên kết header_table
                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    link_node = header_table[item][1]
                    while link_node.link:
                        link_node = link_node.link
                    link_node.link = new_node

            current_node = current_node.children[item]
    return root, header_table

# Xử lý thông tin
freq_raw = get_item_frequency(transactions)
filtered_items = filter_items(freq_raw, minSupport, len(transactions))
freq = {item: freq_raw[item] for item in filtered_items}
sorted_trans = sort_transactions(transactions, freq)
fp_tree_root, header_table = build_fp_tree(sorted_trans, freq)

# Vẽ cây FB-tree
fp_tree_root.display()

Null:1
  Bánh mì:4
    Bơ:3
      Sữa:2
    Sữa:1
  Bơ:1
    Sữa:1


## Bước 5: Tìm các prefix paths

In [66]:
def find_prefix_paths(base_item, header_table):
    conditional_patterns = []

    node = header_table[base_item][1]
    while node:
        path = []
        parent = node.parent
        while parent and parent.item != 'Null':
            path.append(parent.item)
            parent = parent.parent
        path.reverse()
        if path:
            conditional_patterns.append((path, node.count))
        node = node.link
    return conditional_patterns

## Bước 6: Xây conditional FP-Tree
**Mục đích:** Xây dựng lại FB-tree với cách duyệt từ Node nhánh cuối lên Node root
*  Trả về các path từ item về gốc, cùng count (pattern base).

In [67]:
def build_conditional_fp_tree(conditional_patterns, min_support_count):
    item_counts = defaultdict(int)
    for path, count in conditional_patterns:
        for item in path:
            item_counts[item] += count
    freq_items = {item: count for item, count in item_counts.items() if count >= min_support_count}
    if not freq_items:
        return None, None

    root = FPTreeNode('Null', 1)
    header_table = {item: [freq_items[item], None] for item in freq_items}

    for path, count in conditional_patterns:
        filtered_path = [item for item in path if item in freq_items]
        sorted_path = sorted(filtered_path, key=lambda x: (-freq_items[x], x))
        current_node = root
        for item in sorted_path:
            if item in current_node.children:
                current_node.children[item].increment(count)
            else:
                new_node = FPTreeNode(item, count)
                new_node.parent = current_node
                current_node.children[item] = new_node

                if header_table[item][1] is None:
                    header_table[item][1] = new_node
                else:
                    link_node = header_table[item][1]
                    while link_node.link:
                        link_node = link_node.link
                    link_node.link = new_node

            current_node = current_node.children[item]
    return root, header_table

## Bước 7: Đệ quy khai thác FP-Tree
**Mục đích:** Đệ quy khai thác itemsets từ conditional tree.
*  

In [68]:
def mine_fp_tree(tree, header_table, prefix, freq_itemsets, min_support_count):
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])
    for item, (count, node) in sorted_items:
        new_freq_set = prefix + [item]
        freq_itemsets[tuple(new_freq_set)] = count

        conditional_patterns = find_prefix_paths(item, header_table)
        cond_tree, cond_header = build_conditional_fp_tree(conditional_patterns, min_support_count)

        if cond_tree:
            mine_fp_tree(cond_tree, cond_header, new_freq_set, freq_itemsets, min_support_count)

## Bước 8: chạy xử lý thuật toán FB-Growth (Chạy thử với toàn bộ dữ liệu)

In [69]:
# Khai thác frequent itemsets
freq_itemsets = {}
mine_fp_tree(fp_tree_root, header_table, [], freq_itemsets, min_support_count)

# Hiển thị kết quả
for itemset, count in sorted(freq_itemsets.items(), key=lambda x: (-len(x[0]), -x[1])):
    print(f"{set(itemset)}: {count}")

{'Sữa', 'Bánh mì', 'Bơ'}: 2
{'Sữa', 'Bánh mì'}: 3
{'Sữa', 'Bơ'}: 3
{'Bánh mì', 'Bơ'}: 3
{'Sữa'}: 4
{'Bánh mì'}: 4
{'Bơ'}: 4


## Bước 9: Tính toán luật kết hợp và Lift
*  Điều kiện phải có transaction có tối thiểu 2 items
*  Dựa vào điều kiện minConfidence để chọn

In [70]:
from itertools import combinations

def generate_association_rules(freq_itemsets, transactions, min_confidence):
    rules = []
    total_trans = len(transactions)

    for itemset in freq_itemsets:
        if len(itemset) < 2:
            continue  # Bỏ qua tập nhỏ hơn 2 item

        itemset_support = freq_itemsets[itemset]

        for i in range(1, len(itemset)):
            for antecedent in combinations(itemset, i):
                antecedent = tuple(sorted(antecedent))
                consequent = tuple(sorted(set(itemset) - set(antecedent)))

                antecedent_support = freq_itemsets.get(antecedent)
                consequent_support = freq_itemsets.get(consequent)

                if antecedent_support and consequent_support:
                    confidence = itemset_support / antecedent_support
                    lift = confidence / (consequent_support / total_trans)
                    support = itemset_support / total_trans

                    if confidence >= min_confidence:
                        rules.append({
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support': round(support, 3),
                            'confidence': round(confidence, 3),
                            'lift': round(lift, 3)
                        })
    return rules

# Gọi hàm
rules = generate_association_rules(freq_itemsets, transactions, minConfidence)
print(rules)

# In ra danh sách các cặp luật kết hợp và kết quả Confidence và Lift
for rule in rules:
    ant = ', '.join(rule['antecedent'])
    cons = ', '.join(rule['consequent'])
    print(f"{ant} => {cons} | Support: {rule['support']}, Confidence: {rule['confidence']}, Lift: {rule['lift']}")

[{'antecedent': ('Sữa',), 'consequent': ('Bánh mì',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}, {'antecedent': ('Bánh mì',), 'consequent': ('Sữa',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}, {'antecedent': ('Sữa',), 'consequent': ('Bơ',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}, {'antecedent': ('Bơ',), 'consequent': ('Sữa',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}, {'antecedent': ('Bơ',), 'consequent': ('Bánh mì',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}, {'antecedent': ('Bánh mì',), 'consequent': ('Bơ',), 'support': 0.6, 'confidence': 0.75, 'lift': 0.938}]
Sữa => Bánh mì | Support: 0.6, Confidence: 0.75, Lift: 0.938
Bánh mì => Sữa | Support: 0.6, Confidence: 0.75, Lift: 0.938
Sữa => Bơ | Support: 0.6, Confidence: 0.75, Lift: 0.938
Bơ => Sữa | Support: 0.6, Confidence: 0.75, Lift: 0.938
Bơ => Bánh mì | Support: 0.6, Confidence: 0.75, Lift: 0.938
Bánh mì => Bơ | Support: 0.6, Confidence: 0.75, Lift: 0.938
